In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd

In [ ]:
import sys
sys.path.append(r'../')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [ ]:
proj_name = 'output_FJSJ_07-03/'

In [ ]:
filename_basic = proj_name+'Basic_info.npy'
info_basic = np.load(filename_basic, allow_pickle='TRUE').item()      # setting dictionary

In [ ]:
key_subworks = ['01-01']
key_subworks = info_basic['key_subworks']
#key_subworks.remove('2013_10')
key_subworks

### read all stations coordinates

In [ ]:
stalistname = info_basic['stalistname']
stainfo = pd.read_excel(stalistname+'.xlsx')
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_all = stainfo['latitude'].tolist() 
lon_all = stainfo['longitude'].tolist()
d_len = info_basic['d_len']

In [ ]:
def FJ2TJ(f,ds_linear,flag_inter):
    T = 1./f[1:]
    dt = flag_inter*(T[-2]-T[-1])
    T1 = np.arange(T[-2],T[1],dt)
    f1 = np.sort(1./T1[::flag_inter])

    ds_linear1 = np.zeros((np.shape(ds_linear)[0],np.shape(f1)[0]))
    for i in range(np.shape(ds_linear)[0]):
        f_inter = interpolate.interp1d(f,ds_linear[i,:],kind='linear')
        ds_linear1[i,:] = f_inter(f1)
    indx = np.argsort(1./f1)
    T1 = 1./f1[indx]
    ds_linear2 = ds_linear1[:,indx]
    return T1,ds_linear2

In [ ]:
#for key_subwork in key_subworks[0:1]:
for key_subwork in key_subworks:
    fig,ax = plt.subplots(3,2,figsize=(15,18))
    # settings for subwork
    dir_stack = info_basic['dir_stack']
    dir_ds = info_basic['dir_ds']
    #d_len = info_basic['d_len']
    dir_image = info_basic['dir_image']
    
    start = info_basic['start'][key_subwork]
    interval = info_basic['interval'][key_subwork]
    flag_time = info_basic['flag_time'][key_subwork]

    # plot area
    stalistname = proj_name+info_basic['stalistname'] +'-'+key_subwork
    stainfo = pd.read_excel(stalistname+'.xlsx')
    nsta = len(stainfo.iloc[:,0])
    StationPairs = GetStationPairs(nsta)
    nPairs = int(len(StationPairs)/2)
    stalist = stainfo['Station'].tolist()
    lat = stainfo['latitude'].tolist() 
    lon = stainfo['longitude'].tolist()
    ax[0][0]=plotlib.plot_area(ax[0][0],lon_all,lat_all,lon,lat)

    # plot ncfs and ncfst

    outname = key_subwork+'_gather_linear.h5'
    ncffile = h5py.File(dir_stack + outname,'r')
    ncfs_sum_linear = ncffile['ncfs'][:]
    r = ncffile['r'][:]
    f = info_basic['f']
    t = info_basic['t']
    ncffile.close()

    outname = key_subwork+'_gather_FPS.h5'
    ncffile = h5py.File(dir_stack + outname,'r')
    ncfs_sum_FPS = ncffile['ncfs'][:]
    ncffile.close()

    
    ncfst_linear = mathlib.freq_time(ncfs_sum_linear)
    ncfst_FPS = mathlib.freq_time(ncfs_sum_FPS)
    title0 = "Linear Stack subarea time for "+key_subwork
    title1 = "Linear Stack subarea frequency for "+key_subwork
    xlim_f = [0,15]
    ax[0][1] = plotlib.plot_ncfs(ax[0][1],f,ncfs_sum_linear,r,title0,xlim_f,0)



    xlim_T = [-1,1]
    ax[1][0] = plotlib.plot_ncfst(ax[1][0],t,ncfst_linear[start::interval],r[start::interval],title0,flag_time,xlim_T,0)

    title0 = "FPS Stack subarea time for "+key_subwork
    ax[1][1] = plotlib.plot_ncfst(ax[1][1],t,ncfst_FPS[start::interval],r[start::interval],title0,flag_time,xlim_T,0)

    
    # plot fj
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_linear = ds['ds_linear'][0]
    ds_FPS = ds['ds_FPS'][0]
    c = info_basic['c']
    ds.close()
    
    title0 = "Linear stack dispersion curve "+str(d_len)+' days'
    ax[2][0] = plotlib.plot_fj(ax[2][0],ds_linear,title0,f,c,xlim_f,0)
    ax[2][0].set_xlim(xlim_f)
    
    title0 = "FPS stack dispersion curve "+str(d_len)+' days'
    ax[2][1] = plotlib.plot_fj(ax[2][1],ds_FPS,title0,f,c,xlim_f,0)
    ax[2][1].set_xlim(xlim_f)

    


    plt.tight_layout()
    plt.savefig(dir_image+key_subwork+'_standard-output.png',dpi=60)